In [1]:
import json

file = "sorted_ids_url_BL_1-10162.json"

with open(file, "r") as f:

    link_list = json.load(f)


print(f"{file} 로드 완료")

sorted_ids_url_BL_1-10162.json 로드 완료


In [2]:
len(link_list)

9

In [4]:
# pip install webdriver-manager selenium beautifulsoup4 lxml

In [3]:
# imports
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import lxml
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import urlparse, urlunparse


In [4]:
driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://nid.naver.com/nidlogin.login")

### 
## 장르
### naver_series_novel_01_로맨스 ~ 1833
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=201")

### naver_series_novel_02_로판 ~ 619
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=207")

### naver_series_novel_03_판타지 ~493
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=202")

### naver_series_novel_04_현판 ~401
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=208")

### naver_series_novel_05_무협 ~218
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=206")

### naver_series_novel_06_미스터리 ~28
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=203")

https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=203&orderTypeCode=star_score&is&isFinished=false

### naver_series_novel_07_라이트노벨 ~39
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205")

https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205&orderTypeCode=star_score&is&isFinished=false

### naver_series_novel_08_BL ~401
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=209")

In [12]:
driver.get("https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=201")

In [13]:
# # 조회수 처리 함수
# def view_tran(view):
#     view.replace(",", "")
#     if view[-1] in "억":
#         new_view = float(view[:-1])
#         new_view *= 100000000
#     elif view[-1] == "만":
#         new_view = float(view[:-1])
#         new_view *= 10000
#     else:
#         new_view = view[:-1]
#     return int(new_view)

In [14]:

# 크롤링 함수 정의
def crawling(url):
    series_novel = {}
    URL = url
    driver.get(URL)

    series_novel["type"] = "웹소설"
    series_novel["platform"] = "네이버 시리즈"

    id_element = url.split('productNo=')[-1]
    id_element = int(id_element) 
    
    series_novel["id"] = id_element
    
    try:
        title_selector = r"#content > div.end_head > h2"
        title_element = driver.find_element(By.CSS_SELECTOR, title_selector).text
    except:
        title_element = "-"
    series_novel["title"] = title_element


    try:
        status_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(1) > span"
        status_element = driver.find_element(By.CSS_SELECTOR, status_selector).text
    except:
        status_element = "-"
    series_novel["status"] = status_element
    

    try:
        thumbnail_selector = r"#container > div.aside.NE\\=a\\:nvi > span.pic_area > img"
        thumbnail_element = driver.find_element(By.CSS_SELECTOR, thumbnail_selector).get_attribute("src")
    except:
        try:
            thumbnail_selector = r'//*[@id="container"]/div[1]/a/img'
            thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
        except:
            try:
                thumbnail_selector = r'//*[@id="container"]/div[1]/span/img'
                thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
            except:
                thumbnail_element = "-"
    series_novel["thumbnail"] = thumbnail_element


    try:
        genre_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(2) > span > a"
        genre_element = driver.find_element(By.CSS_SELECTOR, genre_selector).text
    except:
        genre_element = "-"
    series_novel["genre"] = genre_element


    try:
        views_selector = r"-"
        views_element = driver.find_element(By.CSS_SELECTOR, views_selector).text
    except:
        views_element = "-"
    series_novel["views"] = views_element


    try:
        rating_selector = r"#content > div.end_head > div.score_area > em"
        rating_element = driver.find_element(By.CSS_SELECTOR, rating_selector).text
        rating_element = float(rating_element)
    except:
        rating_element = "-"
    series_novel["rating"] = rating_element


    try:
        like_selector = r"#content > div.end_head > div.user_action_area > ul > li:nth-child(2) > div > a > em"
        like_element = driver.find_element(By.CSS_SELECTOR, like_selector).text
        like_element = int(like_element.replace(",", ""))   
    except:
        like_element = "-"
    series_novel["like"] = like_element


    try:
        CLICKER_SELECTOR = "#content > div.end_dsc > div:nth-child(1) > span > a"
        driver.find_element(By.CSS_SELECTOR, CLICKER_SELECTOR).click()
        synopsis_selector = "#content > div.end_dsc.open > div:nth-child(2)"
        synopsis_element = driver.find_element(By.CSS_SELECTOR, synopsis_selector).text
    except:
        try:
            synopsis_selector = r'//*[@id="content"]/div[2]/div'
            synopsis_element = driver.find_element(By.XPATH, synopsis_selector).text
        except:
            synopsis_element = "-"
    series_novel["synopsis"] = synopsis_element

    try:
        keywords_selector = r"-"
        keywords_element = driver.find_element(By.CSS_SELECTOR, keywords_selector).text
        keywords_element = keywords_element.replace("#", "")
        keywords_element = keywords_element.split("\n")
    except:
        keywords_element = "-"
    series_novel["keywords"] = keywords_element


    try:
        author_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(3) > a"
        author_element = driver.find_element(By.CSS_SELECTOR, author_selector).text
    except:
        author_element = "-"
    series_novel["author"] = author_element


    try:
        illustrator_selector = r"-"
        illustrator_element = driver.find_element(By.CSS_SELECTOR, illustrator_selector).text
    except:
        illustrator_element = "-"
    series_novel["illustrator"] = illustrator_element


    try:                     
        age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
        age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
    except:
        try:
            age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
            age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
            exclude_keywords = ["그림", "출판사"]
            if not any(keyword in  age_rating_element for keyword in exclude_keywords):
                age_rating_element = int(age_rating_element)
        except:
            try:
                age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(5)"
                age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
                exclude_keywords = ["그림", "출판사"]
                if not any(keyword in  age_rating_element for keyword in exclude_keywords):
                    age_rating_element = int(age_rating_element)
                else:
                    age_rating_element = "-"
            except:
                try:
                    age_rating_selector = r'//*[@id="content"]/ul[1]/li/ul/li[5]' 
                    age_rating_element = driver.find_element(By.XPATH, age_rating_selector).text
                    exclude_keywords = ["그림", "출판사"]
                except:
                    age_rating_element = "-"
    series_novel["age_rating"] = age_rating_element
    
    time.sleep(2)
    try:
        first_view_button = driver.find_element(By.CSS_SELECTOR, "#content > table > thead > tr > th.firstview.v2 > div > button")
        first_view_button.click()
        time.sleep(1)  # 1화 페이지 로딩 대기
    except:
        try:
            first_view_button = driver.find_element(By.CSS_SELECTOR, "#content > table > thead > tr > th.firstview.v2 > div > button")
            first_view_button.click()
            time.sleep(1)  # 1화 페이지 로딩 대기   
        except:
            pass
        
    try:
        date_element = driver.find_element(By.CSS_SELECTOR, "#volumeList > tr._volume_row_1 > td.subj > em")
        date_element = date_element.text.strip("()")
    except:
        try:
            # 1화 날짜 정보 가져오기
            date_element = driver.find_element(By.CSS_SELECTOR, "#volumeList > tr._volume_row_1 > td.subj > em")
            date_element = date_element.text.strip("()")  # "(2025.02.19.)"에서 괄호 제거
        except:
            date_element = "-"
    series_novel["first_episode"] = date_element
    
    time.sleep(1)
    try:    
        comments_selector = r"#cbox_module > div > div.u_cbox_head > span"
        comments_element = driver.find_element(By.CSS_SELECTOR, comments_selector).text
        comments_element = int(comments_element.replace(",", ""))
    except:
        comments_element = "-"
    series_novel["comments"] = comments_element
    

    free_selector = r"#content > div.end_title_fixed_price_sign.NE\=a\:nvi > div > div.area_text_information > strong"
    wait_selector = r"#container > div.aside.NE\=a\:nvi > span > em.ico_end_head.daily_free2"
    price_selector = r"#content > div.area_ebook_price_information > div > dl > dd > div > div.area_price"

    try:
        driver.find_element(By.CSS_SELECTOR, free_selector)
        price_element = driver.find_element(By.CSS_SELECTOR, free_selector).text.get_attribute("무료")
    except:
        try:
            driver.find_element(By.CSS_SELECTOR, wait_selector)
            price_element = driver.find_element(By.CSS_SELECTOR, wait_selector).text
        except:
            try:
                price_element = driver.find_element(By.CSS_SELECTOR, price_selector).text
            except:
                price_element = "-"
    series_novel["price"] = price_element
    

    try:
        episode_selector = r"#content > h5 > strong"
        episode_element = driver.find_element(By.CSS_SELECTOR, episode_selector).text
        episode_element["episode"] = int(episode_element)
    except:
        try:
            episode_selector = "#content > h5.end_total_episode > strong"
            episode_element = driver.find_element(By.CSS_SELECTOR, episode_selector).text
            episode_element = int(episode_element)
        except:
            episode_element = "-"
    series_novel["episode"] = episode_element
    
    URL = url + "?tab_type=overview"
    driver.get(URL)
    series_novel["url"] = url
    
    return series_novel


In [15]:
naver_series_novel = []

In [16]:
for url in link_list:
    series_novel = crawling(url)
    naver_series_novel.append(series_novel)
    print(url)

InvalidArgumentException: Message: invalid argument: 'url' must be a string
  (Session info: chrome=133.0.6943.142)
Stacktrace:
	GetHandleVerifier [0x008F0B43+25139]
	(No symbol) [0x008813F4]
	(No symbol) [0x007604E3]
	(No symbol) [0x007EF4EE]
	(No symbol) [0x007CCFCC]
	(No symbol) [0x007EE778]
	(No symbol) [0x007CCDC6]
	(No symbol) [0x0079BDE9]
	(No symbol) [0x0079D124]
	GetHandleVerifier [0x00BF4373+3185251]
	GetHandleVerifier [0x00C1291A+3309578]
	GetHandleVerifier [0x00C0CF42+3286578]
	GetHandleVerifier [0x00987AE0+643536]
	(No symbol) [0x0088A20D]
	(No symbol) [0x008870B8]
	(No symbol) [0x00887257]
	(No symbol) [0x00879E00]
	BaseThreadInitThunk [0x76555D49+25]
	RtlInitializeExceptionChain [0x7739CDEB+107]
	RtlGetAppContainerNamedObjectPath [0x7739CD71+561]


In [11]:
# import json

# with open("total_url_ROMACE_detail.json", "w", encoding="utf-8") as f:
#     json.dump(naver_series_novel, f, ensure_ascii=False, indent=4)

In [8]:
import json

naver_series_novel = []
save_interval = 100  # 50개마다 저장

start_index = 1  # 51번째부터 크롤링 시작

def save_progress(data, index):
    filename = f"ns_wn_WUXIA_{index}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Progress saved in {filename}")

for idx, url in enumerate(link_list[start_index - 1:], start=start_index):
    series_novel = crawling(url)
    naver_series_novel.append(series_novel)
    print(url)
    
    if (idx - start_index + 1) % save_interval == 0:
        save_progress(naver_series_novel, idx)
        naver_series_novel = []  # 저장 후 리스트 초기화

# 마지막 저장 (남은 데이터 저장)
if naver_series_novel:
    save_progress(naver_series_novel, len(link_list))
    print("Final progress saved.")

https://series.naver.com/novel/detail.series?productNo=10611913
https://series.naver.com/novel/detail.series?productNo=11674416
https://series.naver.com/novel/detail.series?productNo=11047926
https://series.naver.com/novel/detail.series?productNo=10443405
https://series.naver.com/novel/detail.series?productNo=11162168
https://series.naver.com/novel/detail.series?productNo=3030120
https://series.naver.com/novel/detail.series?productNo=12100744
https://series.naver.com/novel/detail.series?productNo=10869743
https://series.naver.com/novel/detail.series?productNo=11595091
https://series.naver.com/novel/detail.series?productNo=11610478
https://series.naver.com/novel/detail.series?productNo=11156818
https://series.naver.com/novel/detail.series?productNo=10131826
https://series.naver.com/novel/detail.series?productNo=11405703
https://series.naver.com/novel/detail.series?productNo=11396984
https://series.naver.com/novel/detail.series?productNo=9779791
https://series.naver.com/novel/detail.seri

In [ ]:

### 50개씩 저장하는 코드  url 순서 지정 없음

import json

naver_series_novel = []
save_interval = 50  # 50개마다 저장

def save_progress(data, index):
    filename = f"ns_wn_ROMANCE_{index}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Progress saved in {filename}")

for idx, url in enumerate(link_list, start=1):
    series_novel = crawling(url)
    naver_series_novel.append(series_novel)
    print(url)
    
    if idx % save_interval == 0:
        save_progress(naver_series_novel, idx)

# 마지막 저장 (남은 데이터 저장)
save_progress(naver_series_novel, len(link_list))
print("Final progress saved.")

In [ ]:
naver_series_novel

[{'type': '웹소설',
  'platform': '네이버 시리즈',
  'title': '칭찬스티커 받는 단종',
  'status': '연재중',
  'thumbnail': 'https://comicthumb-phinf.pstatic.net/20240216_93/pocket_1708064631462FdlQI_JPEG/%C4%AA%C2%F9_%BD%BA%C6%BC%C4%BF_%B9%DE%B4%C2_%B4%DC%C1%BE_%C5%B8%C0%CC%C6%B2_%C3%D6%C1%BE.jpg?type=m260',
  'genre': '판타지',
  'views': '-',
  'rating': 9.8,
  'like': 21,
  'synopsis': '[세종 이도가 칭찬스티커를 선물합니다.]\n\n[문종 이향이 칭찬스티커를 선물합니다.]\n[수양대군 이유가 칭찬스티커를 선물합니다.]\n…….\n[칭찬스티커북을 완성했습니다. 원하는 선물을 말씀해주세요.]\n[추상적인 선물은 받을 수 없습니다. ex) 동생 만들어주세요.]\n[한 손으로 들 수 없는 선물은 받을 수 없습니다. ex) 핵무기]\n"영조표 게장?"\n접기',
  'keywords': '-',
  'author': '문환',
  'illustrator': '-',
  'age_rating': '전체 이용가',
  'price': '쿠키 1개',
  'episode': 337,
  'url': 'https://series.naver.com/novel/detail.series?productNo=10773837'},
 {'type': '웹소설',
  'platform': '네이버 시리즈',
  'title': '왕자가 친구를 사귀는 방법',
  'status': '연재중',
  'thumbnail': 'https://comicthumb-phinf.pstatic.net/20241211_292/pocket_1733889099579ca9li_JPEG/%BD%C3%B8%AE%C1%EE_%BC%BC%B7%CE.jpg

In [25]:
import json

def find_url_index(filename, target_url):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        for idx, item in enumerate(data, start=1):
            if item == target_url:
                return idx
        
        return None  # URL이 존재하지 않음
    
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON in {filename}.")
        return None
filename = "webnovel_naver_series_01.json"  # 탐색할 JSON 파일
target_url = "https://series.naver.com/novel/detail.series?productNo=11529513"  # 찾을 URL

index = find_url_index(filename, target_url)

if index:
    print(f"URL found at position {index}.")
else:
    print("URL not found.")

URL found at position 2870.


In [ ]:
import json

def load_json(path: str):
    """
    JSON 파일에서 데이터를 불러옵니다.
    """
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# # 저장된 파일 경로
file_path = "webnovel_naver_series_01_detail.json"
# file_path = "webnovel_naver_series_02_detail.json"
# file_path = "webnovel_naver_series_03_detail.json"
# file_path = "webnovel_naver_series_04_detail.json"
# file_path = "webnovel_naver_series_05_detail.json"
# file_path = "webnovel_naver_series_06_detail.json"
# file_path = "webnovel_naver_series_07_detail.json"
# file_path = "webnovel_naver_series_08_detail.json"

# pickle 데이터 로드
loaded_data = load_json(file_path)

# 로드된 데이터 출력
print(f"로드된 데이터 개수: {len(loaded_data)}개")
for url in loaded_data:
    print(url)

FileNotFoundError: [Errno 2] No such file or directory: 'webnovel_naver_series_01_detail.json'

: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urlunparse

from dotenv import load_dotenv
import logging
import pickle
import json
import time
import re
import os

load_dotenv()
naver_id = os.getenv("NAVER_ID")
naver_pw = os.getenv("NAVER_PWD")
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://nid.naver.com/nidlogin.login")

# URL to crawl
url_ids = [
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=201",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=207",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=202",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=208",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=206",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=203",
    "https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=203&orderTypeCode=star_score&is&isFinished=false",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=205",
    "https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205&orderTypeCode=star_score&is&isFinished=false",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=209",
]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
req = urllib.request.Request(url, headers=headers)
sourcecode = urllib.request.urlopen(req).read()
soup = BeautifulSoup(sourcecode, "html.parser")

# 소설 URL 추출
seriesnovel_urls = []
# 주간 top list 크롤링
for li in soup.select('div.comic_lst_thum_wrap ul.comic_top_lst li'): 
    link = li.find('a', class_='pic')
# 장르내 평점순 크롤링
for li in soup.select('div.lst_thum_wrap ul.lst_list li'):  # 카테고리 장르 내에서 평점순 크롤링
    link = li.find('a', class_='pic')
    if link and "href" in link.attrs:
        href_value = link["href"]
        # 필터링: "categoryProductList" 포함 링크 제외
        if "categoryProductList.series" not in href_value:
            full_url = "https://series.naver.com" + href_value
            seriesnovel_urls.append(full_url)

def crawling(url):
    # Initialize the WebDriver
    driver = webdriver.Chrome()  # Or use another browser driver as needed
    
    # Navigate to the URL
    driver.get(url)

    series_novel = {}
    series_novel["type"] = "웹소설"
    series_novel["platform"] = "네이버 시리즈"
    
    
    try:                     
        age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
        age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
    except:
        try:
            age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
            age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
            exclude_keywords = ["그림", "출판사"]
            if not any(keyword in  age_rating_element for keyword in exclude_keywords):
                age_rating_element = int(age_rating_element)
        except:
                age_rating_element = "-"
    series_novel["age_rating"] = age_rating_element


    try:
        thumbnail_selector = r"#container > div.aside.NE\\=a\\:nvi > span.pic_area > img"
        thumbnail_element = driver.find_element(By.CSS_SELECTOR, thumbnail_selector).get_attribute("src")
    except:
        try:
            thumbnail_selector = r'//*[@id="container"]/div[1]/a/img'
            thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
        except:
            try:
                thumbnail_selector = r'//*[@id="container"]/div[1]/span/img'
                thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
            except:
                thumbnail_element = "-"
    series_novel["thumbnail"] = thumbnail_element
    
    
    try:
        CLICKER_SELECTOR = "#content > div.end_dsc > div:nth-child(1) > span > a"
        driver.find_element(By.CSS_SELECTOR, CLICKER_SELECTOR).click()
        synopsis_selector = "#content > div.end_dsc.open > div:nth-child(2)"
        synopsis_element = driver.find_element(
            By.CSS_SELECTOR, synopsis_selector
        ).text
    except:
        try:
            synopsis_selector = r'//*[@id="content"]/div[2]/div'
            synopsis_element = driver.find_element(By.XPATH, synopsis_selector).text
        except:
            synopsis_element = "-"
    series_novel["synopsis"] = synopsis_element


    try:
        like_selector = r"#content > div.end_head > div.user_action_area > ul > li:nth-child(2) > div > a > span > em"
        like_element = driver.find_element(By.CSS_SELECTOR, like_selector).text
        like_element = int(like_element.replace(",", ""))
    except:
        try:
            like_selector = r'//*[@id="content"]/div[1]/div[2]/ul/li[2]/div/a/em'
            like_element = driver.find_element(By.XPATH, like_selector).text
            like_element = int(like_element.replace(",", ""))
        except:
            like_element = "-"
    series_novel["like"] = like_element



    # Close the browser
    driver.quit()

    return series_novel


# Call the crawling function
result = crawling(url)

# Print the result
print("age_rating_element:", result["age_rating"])
print("thumbnail_element:", result["thumbnail"])
print("description_element:", result["description"])
print("liken_element:", result["like"])


전체 이용가
age_rating_element: -
thumbnail_element: https://comicthumb-phinf.pstatic.net/20210721_137/pocket_1626854955343mN2P4_JPEG/650NAVER-.jpg?type=m260
description_element: 여섯 번째 남편이 건넨 독을 마시고 황녀 리엔은 죽음을 맞이했다.

 “그대는 황제의 수단에 불과했어.” 

다시 눈을 떴을 때는 다섯 번째 남편의 장례식날로 돌아와 있었다.

리엔은 복수를 다짐했다. 
그리고 그 복수를 위해 얼음보다 차갑고 잔혹한 대공, 클로시안을 찾아갔다. 
3년간의 계약 결혼. 그리고 복수만을 위해 손을 잡은 남자. 그런데. 

“계약서의 존재는 잊으세요.” 

계약 조항 하나하나를 지우고 수정하던 클로시안은 급기야 계약 결혼에서 가장 중요한 계약서를 잊고 지내라고 했다. 

이 남자, 이게 계약 결혼이라는 자각은 있는 걸까?
접기
liken_element: 74


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urlunparse

from dotenv import load_dotenv
import logging
import pickle
import json
import time
import re
import os

load_dotenv()
naver_id = os.getenv("NAVER_ID")
naver_pw = os.getenv("NAVER_PWD")
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://nid.naver.com/nidlogin.login")

# URL to crawl
url_ids = [
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=201",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=207",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=202",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=208",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=206",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=203",
    "https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=203&orderTypeCode=star_score&is&isFinished=false",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=205",
    "https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205&orderTypeCode=star_score&is&isFinished=false",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=209",
]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
req = urllib.request.Request(url, headers=headers)
sourcecode = urllib.request.urlopen(req).read()
soup = BeautifulSoup(sourcecode, "html.parser")

# 소설 URL 추출
seriesnovel_urls = []
# 주간 top list 크롤링
for li in soup.select('div.comic_lst_thum_wrap ul.comic_top_lst li'): 
    link = li.find('a', class_='pic')
# 장르내 평점순 크롤링
for li in soup.select('div.lst_thum_wrap ul.lst_list li'):  # 카테고리 장르 내에서 평점순 크롤링
    link = li.find('a', class_='pic')
    if link and "href" in link.attrs:
        href_value = link["href"]
        # 필터링: "categoryProductList" 포함 링크 제외
        if "categoryProductList.series" not in href_value:
            full_url = "https://series.naver.com" + href_value
            seriesnovel_urls.append(full_url)

def crawling(url):
    # Initialize the WebDriver
    driver = webdriver.Chrome()  # Or use another browser driver as needed
    
    # Navigate to the URL
    driver.get(url)

    series_novel = {}
    series_novel["type"] = "웹소설"
    series_novel["platform"] = "네이버 시리즈"
    
    
    try:                     
        age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
        age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
    except:
        try:
            age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
            age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
            exclude_keywords = ["그림", "출판사"]
            if not any(keyword in  age_rating_element for keyword in exclude_keywords):
                age_rating_element = int(age_rating_element)
        except:
                age_rating_element = "-"
    series_novel["age_rating"] = age_rating_element


    try:
        thumbnail_selector = r"#container > div.aside.NE\\=a\\:nvi > span.pic_area > img"
        thumbnail_element = driver.find_element(By.CSS_SELECTOR, thumbnail_selector).get_attribute("src")
    except:
        try:
            thumbnail_selector = r'//*[@id="container"]/div[1]/a/img'
            thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
        except:
            try:
                thumbnail_selector = r'//*[@id="container"]/div[1]/span/img'
                thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
            except:
                thumbnail_element = "-"
    series_novel["thumbnail"] = thumbnail_element
    
    
    try:
        CLICKER_SELECTOR = "#content > div.end_dsc > div:nth-child(1) > span > a"
        driver.find_element(By.CSS_SELECTOR, CLICKER_SELECTOR).click()
        synopsis_selector = "#content > div.end_dsc.open > div:nth-child(2)"
        synopsis_element = driver.find_element(
            By.CSS_SELECTOR, synopsis_selector
        ).text
    except:
        try:
            synopsis_selector = r'//*[@id="content"]/div[2]/div'
            synopsis_element = driver.find_element(By.XPATH, synopsis_selector).text
        except:
            synopsis_element = "-"
    series_novel["synopsis"] = synopsis_element


    try:
        like_selector = r"#content > div.end_head > div.user_action_area > ul > li:nth-child(2) > div > a > span > em"
        like_element = driver.find_element(By.CSS_SELECTOR, like_selector).text
        like_element = int(like_element.replace(",", ""))
    except:
        try:
            like_selector = r'//*[@id="content"]/div[1]/div[2]/ul/li[2]/div/a/em'
            like_element = driver.find_element(By.XPATH, like_selector).text
            like_element = int(like_element.replace(",", ""))
        except:
            like_element = "-"
    series_novel["like"] = like_element



    # Close the browser
    driver.quit()

    return series_novel


# Call the crawling function
result = crawling(url)

# Print the result
print("age_rating_element:", result["age_rating"])
print("thumbnail_element:", result["thumbnail"])
print("description_element:", result["description"])
print("liken_element:", result["like"])


전체 이용가
age_rating_element: -
thumbnail_element: https://comicthumb-phinf.pstatic.net/20210721_137/pocket_1626854955343mN2P4_JPEG/650NAVER-.jpg?type=m260
description_element: 여섯 번째 남편이 건넨 독을 마시고 황녀 리엔은 죽음을 맞이했다.

 “그대는 황제의 수단에 불과했어.” 

다시 눈을 떴을 때는 다섯 번째 남편의 장례식날로 돌아와 있었다.

리엔은 복수를 다짐했다. 
그리고 그 복수를 위해 얼음보다 차갑고 잔혹한 대공, 클로시안을 찾아갔다. 
3년간의 계약 결혼. 그리고 복수만을 위해 손을 잡은 남자. 그런데. 

“계약서의 존재는 잊으세요.” 

계약 조항 하나하나를 지우고 수정하던 클로시안은 급기야 계약 결혼에서 가장 중요한 계약서를 잊고 지내라고 했다. 

이 남자, 이게 계약 결혼이라는 자각은 있는 걸까?
접기
liken_element: 74


In [10]:
# imports
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import lxml
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import urlparse, urlunparse

driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://nid.naver.com/nidlogin.login")

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import lxml
import time

# # Selenium WebDriver 설정
# driver = webdriver.Chrome()




# 크롤링 함수 정의
def crawling(url):
    series_novel = {}
    
    # 해당 URL로 이동
    driver.get(url)

    time.sleep(2)
    try:
        first_view_button = driver.find_element(By.CSS_SELECTOR, "#content > table > thead > tr > th.firstview.v2 > div > button")
        first_view_button.click()
        time.sleep(1)  # 1화 페이지 로딩 대기
    except:
        try:
            first_view_button = driver.find_element(By.CSS_SELECTOR, "#content > table > thead > tr > th.firstview.v2 > div > button")
            first_view_button.click()
            time.sleep(1)  # 1화 페이지 로딩 대기   
        except:
            pass
        
    try:
        date_element = driver.find_element(By.CSS_SELECTOR, "#volumeList > tr._volume_row_1 > td.subj > em")
        date_element = date_element.text.strip("()")
    except:
        try:
            # 1화 날짜 정보 가져오기
            date_element = driver.find_element(By.CSS_SELECTOR, "#volumeList > tr._volume_row_1 > td.subj > em")
            date_element = date_element.text.strip("()")  # "(2025.02.19.)"에서 괄호 제거
        except:
            date_element = "-"
    series_novel["first_episode"] = date_element
    return series_novel

# URL 설정
url = "https://series.naver.com/novel/detail.series?productNo=11392564"

# 크롤링 실행
novel_data = crawling(url)

# 결과 출력
print(novel_data)

# WebDriver 종료
driver.quit()


{'first_episode': '2024.08.01.'}
